In [10]:
import pymongo #type: ignore
import pyodbc #type: ignore

In [11]:
# Khai báo thông tin kết nối
stringConectionSqlServer = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=127.0.0.1;DATABASE=master;UID=sa;PWD=111111'
stringConectionMongo = 'mongodb://localhost:27017'

# Khai báo tên database
dbName = 'HOADON'

# Danh sách từ khóa nếu tên bảng chứa nó thì loại bỏ
listKeyword = ['merge', 'snapshot', 'repl', 'diagram']

In [12]:
# Kết nối Sql Server
conn_sql = pyodbc.connect(stringConectionSqlServer)

In [13]:
# Kết nối đến MongoDB
client = pymongo.MongoClient(stringConectionMongo)

In [14]:
# Trích xuất dữ liệu từ SQL Server
cursor = conn_sql.cursor()
cursor.execute("SELECT * FROM " + dbName + ".INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'")
listTable = [table[2] for table in cursor.fetchall() if not any(keyword in table[2].lower() for keyword in listKeyword)]
listTable

['KHACHHANG', 'MUCTIENGIO', 'PHONG', 'DICHVU', 'HOADON', 'CHITIET_SUDUNGDV']

In [15]:
# Lấy database trên MongoDB theo tên
db = client[dbName]

In [16]:
# Tạo collection trên MongoDB
for table in listTable:
    if table not in db.list_collection_names():
        db.create_collection(table)
        print("Tạo collection " + table + " thành công")
    else:
        print("Collection " + table + " đã tồn tại")

Tạo collection KHACHHANG thành công
Tạo collection MUCTIENGIO thành công
Tạo collection PHONG thành công
Tạo collection DICHVU thành công
Tạo collection HOADON thành công
Tạo collection CHITIET_SUDUNGDV thành công


In [17]:
# Insert dữ liệu vào MongoDB
for table in listTable:

    fields = cursor.execute("SELECT COLUMN_NAME FROM " + dbName + ".INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'" + table + "'").fetchall()
    primaryKeys = cursor.execute("SELECT COLUMN_NAME FROM " + dbName + ".INFORMATION_SCHEMA.KEY_COLUMN_USAGE WHERE TABLE_NAME = '" + table + "'").fetchall()
    rows = cursor.execute("SELECT * FROM " + dbName + ".dbo." + table).fetchall()

    for row in rows:
        # Tạo document từ row của SQL Server
        document = {fields[i][0]: row[i] for i in range(len(row)) if fields[i][0] != 'rowguid'}

        # Nếu document đã tồn tại thì bỏ qua
        if db[table].find_one(document) is not None:
            print("Document already exists.")
            continue

        # Nếu document chỉ có khóa chính giống nhau thì cập nhật document
        elif db[table].find_one({key[0]: document[key[0]] for key in primaryKeys}) is not None:
            db[table].update_one({key[0]: document[key[0]] for key in primaryKeys}, {'$set': document})
            print("Updated document:", document)

        # Thêm document mới
        else:
            db[table].insert_one(document)
            print("Inserted document:", document)

Inserted document: {'MaKH': 'KH001', 'TenKH': 'Tran Van Nam', 'DiaChi': 'Hai Chau', 'SoDT': '0905123456', 'MaSoThue': '12345678', '_id': ObjectId('663c810db11cfd91c8ad86d2')}
Inserted document: {'MaKH': 'KH002', 'TenKH': 'Nguyen Mai Anh', 'DiaChi': 'Lien Chieu', 'SoDT': '0905123456', 'MaSoThue': '12345679', '_id': ObjectId('663c810db11cfd91c8ad86d3')}
Inserted document: {'MaKH': 'KH003', 'TenKH': 'Phan Hoai Lan Khue', 'DiaChi': 'Hoa Vang', 'SoDT': '0905123456', 'MaSoThue': '12345680', '_id': ObjectId('663c810db11cfd91c8ad86d4')}
Inserted document: {'MaKH': 'KH004', 'TenKH': 'Nguyen Hoai Nguyen', 'DiaChi': 'Hoa Cam', 'SoDT': '0905123456', 'MaSoThue': '12345681', '_id': ObjectId('663c810db11cfd91c8ad86d5')}
Inserted document: {'MaKH': 'KH005', 'TenKH': 'Le Truong Ngoc Anh', 'DiaChi': 'Hai Chau', 'SoDT': '0905123456', 'MaSoThue': '12345682', '_id': ObjectId('663c810db11cfd91c8ad86d6')}
Inserted document: {'MaKH': 'KH006', 'TenKH': 'Ho Hoai Anh', 'DiaChi': 'Hai Chau', 'SoDT': '0905123456',

In [18]:
# Đóng kết nối
conn_sql.close()
client.close()